In [1]:
import pandas as pd
from sqlite3 import connect
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.cluster import KMeans
from sklearn import preprocessing
from keras.utils.np_utils import to_categorical


In [2]:
conn = connect("data/dataset_review.db")
dataset = pd.read_sql("SELECT * FROM games", conn)
conn.close()

dataset.set_index("index", inplace=True)
dataset = dataset.loc[dataset["summary"].notnull()]
dataset.reset_index(drop=True, inplace=True)

In [3]:
# tfv = CountVectorizer(analyzer="word", stop_words = 'english')

# tfv.fit(dataset["summary"].str.lower())
# summary = tfv.transform(dataset["summary"])
# kmeans = KMeans(n_clusters=10, random_state=7).fit(summary)
# np.save("data/summary_kmeans.npy", np.array(kmeans.labels_))

summary_kmeans = np.load("data/summary_kmeans.npy", allow_pickle=True)

In [4]:
categorical_labels = to_categorical(summary_kmeans, num_classes=10)
dataset = pd.concat([dataset, pd.DataFrame(categorical_labels, columns=["summary_kmean_" + str(i) for i in range(10)])], axis=1)

In [5]:
print(dataset.head())

                                   name       platform  \
0  The Legend of Zelda: Ocarina of Time    Nintendo_64   
1              Tony Hawk's Pro Skater 2    PlayStation   
2                   Grand Theft Auto IV  PlayStation_3   
3                           SoulCalibur      Dreamcast   
4                   Grand Theft Auto IV       Xbox_360   

                                             summary  meta_score  user_review  \
0  As a young boy, Link is tricked by Ganondorf, ...        0.99         0.91   
1  As most major publishers' development efforts ...        0.98         0.74   
2  [Metacritic's 2008 PS3 Game of the Year; Also ...        0.98         0.77   
3  This is a tale of souls and swords, transcendi...        0.98         0.84   
4  [Metacritic's 2008 Xbox 360 Game of the Year; ...        0.98         0.79   

   publish_year  publish_month  \
0          1998             11   
1          2000              9   
2          2008              4   
3          1999             